In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.ticker import AutoMinorLocator
from matplotlib.gridspec import GridSpec
from sklearn.datasets import load_iris

In [2]:
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
sns.set(style='whitegrid', context='notebook')

In [3]:
# 01. Загрузка данных и быстрый взгляд
# 02. Приведение типов, базовая чистка
# 03. Анализ пропусков, заполнение
# 04. Создание вспомогательных фичей
# 05. Базовые описательные статистики (Stage 2)
# 06. Выбросы: диагностика и фильтрация
# 07. Корреляции и scatterplots (Stage 3, уже на очищенных данных)
# 08. Локальный анализ по пригородам/регионам (Stage 4)
# 09. Финальные выводы и гипотезы (Stage 5, текстом)
# 10. (Опционально) Приложение: проверка устойчивости результатов (до/после чистки)

In [4]:
# 01. Загрузка данных и быстрый взгляд

In [5]:
# df_houses = pd.read_csv('/Users/vladislavlipkin/Downloads/melb_data.csv') # MAC OS
df_houses = pd.read_csv(r'C:\Users\Incognitus\Downloads\melb_data.csv') # WINDOWS

In [6]:
# df_houses.head(10)
# df_houses.shape
df_houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

In [7]:
# просмотр таблицы , топ 5 дорогих объектов
df_houses.sort_values(by='Price', ascending=False).head(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
12094,Mulgrave,35 Bevis St,3,h,"9,000,000.00",PI,Hall,29/07/2017,18.80,"3,170.00",...,1.00,1.00,744.00,117.00,"1,960.00",Monash,-37.93,145.16,South-Eastern Metropolitan,"7,113.00"
7692,Canterbury,49 Mangarra Rd,5,h,"8,000,000.00",VB,Sotheby's,13/05/2017,9.00,"3,126.00",...,5.00,4.00,"2,079.00",464.30,"1,880.00",Boroondara,-37.82,145.07,Southern Metropolitan,"3,265.00"
9575,Hawthorn,49 Lisson Gr,4,h,"7,650,000.00",S,Abercromby's,17/06/2017,5.30,"3,122.00",...,2.00,4.00,"1,690.00",284.00,"1,863.00",Boroondara,-37.83,145.03,Southern Metropolitan,"11,308.00"
3616,Kew,15 Barry St,6,h,"6,500,000.00",S,Jellis,13/08/2016,5.60,"3,101.00",...,6.00,3.00,"1,334.00",365.00,"1,890.00",Boroondara,-37.80,145.03,Southern Metropolitan,"10,331.00"
12557,Middle Park,136 Page St,5,h,"6,400,000.00",S,Marshall,9/09/2017,3.00,"3,206.00",...,2.00,1.00,553.00,308.00,"1,920.00",NaN,-37.85,144.96,Southern Metropolitan,"2,019.00"


In [8]:
# 02. Приведение типов, базовая чистка

In [9]:
# преобразовываем дату
df_houses['Date'] = pd.to_datetime(df_houses['Date'], format='%d/%m/%Y', errors='coerce')

# Строковые категории — в category (экономит память, удобнее группировать)
cat_cols = ['Suburb', 'Type', 'Method', 'SellerG', 'CouncilArea', 'Regionname', 'Postcode']
for c in cat_cols:
    if c in df_houses.columns:
        df_houses[c] = df_houses[c].astype('category')

# Создадим копию сырых данных
df_raw = df_houses.copy()

In [11]:
# 03. Анализ пропусков, заполнение

In [13]:
# найти пропущенные строки
df_houses.isnull().sum().sort_values(ascending=False)

CouncilArea      1369
Suburb              0
Bathroom            0
Regionname          0
Longtitude          0
Lattitude           0
YearBuilt           0
BuildingArea        0
Landsize            0
Car                 0
Bedroom2            0
Address             0
Postcode            0
Distance            0
Date                0
SellerG             0
Method              0
Price               0
Type                0
Rooms               0
Propertycount       0
dtype: int64

In [12]:
#  заполнение медианой сначала в Suburb, после в оставшихся NaN общей медианой по столбцу
for col in ['BuildingArea', 'Landsize', 'YearBuilt', 'Car']:
    if col in df_houses.columns:
        # заполнение пустых ячеек по Suburb
        df_houses[col] = df_houses[col].fillna(df_houses.groupby('Suburb')[col].transform('median'))
        # страховка , заполнение по всему столбцу без группировки
        df_houses[col] = df_houses[col].fillna(df_houses[col].median())

C:\Users\Incognitus\AppData\Local\Temp\ipykernel_2864\4030824298.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_houses[col] = df_houses[col].fillna(df_houses.groupby('Suburb')[col].transform('median'))


In [14]:
# 04. Создание вспомогательных фичей

In [15]:
# сделать столбец стоимость более читаемым
df_houses['price_mln'] = df_houses['Price'] / 1_000_000
# бины по расстоянию для BOXPLOT
df_houses['Distance_bin'] = pd.qcut(df_houses['Distance'], q=5, duplicates='drop')